In [1]:
from vocabulary import create_vocabulary, analyze_text, read_text, filter_text, get_vocab, get_tokens
filename = 'suc_ve_ceza_fyodor_mihailovic_dostoyevski.txt'
#filename = 'suc_ve_ceza_onsoz.txt'

replacement_dict = { "’":"'", "‘":"'", "–":"-"} 

text = read_text(filename)
filtered_text = filter_text(text,replacement_dict)
# save filtered text
f = open("filtered_text.txt", "w")
f.write(filtered_text)
f.close()
# compare text and filtered_text
print("text length: ",len(text))
print("filtered_text length: ",len(filtered_text))

# analyze text
punctuations = analyze_text(filtered_text)
print("punctuations length: ",len(punctuations))
print("punctuations: ",punctuations)


create_vocabulary(filtered_text)
vocab = get_vocab()
tokens = get_tokens()
print("vocab length: ",len(vocab))
print("tokens length: ",len(tokens))
print("vocab: ",vocab[:100])
print("tokens: ",tokens[:100])

# save vocab
f = open("vocab.txt", "w")
for word in vocab:
    f.write(word+"\n")
f.close()


!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~”“—‘’…«»–
text length:  1220059
filtered_text length:  1220059
punctuations length:  18
punctuations:  {':': 1669, ',': 15183, '.': 10494, '/': 5, '(': 191, ')': 191, ';': 1242, "'": 2515, '“': 891, '…': 2950, '”': 892, '-': 2489, '?': 2495, '!': 3723, '[': 162, ']': 162, '—': 2772, '*': 3}
vocab length:  28255
tokens length:  214589
vocab:  ['!', "'", '(', ')', '*', ',', '-', '.', '/', '0212', '1', '1.80', '1.90', '10', '1063', '11', '12', '13', '14', '15', '16', '17', '1712', '1778', '1784', '1793', '1795', '1799', '18', '1812', '1813', '1817', '1821', '1829', '1830', '1840', '1844', '1845', '1846', '1847', '1849', '1850', '1854', '1855', '1859', '1860', '1861', '1864', '1865', '1866', '1873', '1878', '1881', '1887', '1894', '19', '1941', '2', '20', '2006', '21', '22', '23', '24', '25', '252', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '34430', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49'

In [54]:
#create vocabulary

# make words unique to reduce vocab size
vocab = set(words)
print(len(vocab))
print(vocab)
# add punctuation back in to vocabulary
vocab.update(punctuation)
print(len(vocab))
# search for specific words in the vocabulary
search_words = ['suc', 'ceza', 'dostoyevski', 'mihailovic', 'fyodor','…','balzac','ın',' ']
for word in search_words:
    if word in vocab:
        print('Found: %s' % word)
    else:
        print('Not Found: %s' % word)

# convert words to integers
vocab = sorted(vocab)
print(vocab)
print(len(vocab))
# create a map of words to integers
vocab_to_int = dict((c, i) for i, c in enumerate(vocab))
print(vocab_to_int)
# integer encode text
integer_encoded = [vocab_to_int[word] for word in words]
print(integer_encoded[:100])
# reverse map of integers to words
int_to_vocab = dict((i, c) for i, c in enumerate(vocab))
print(int_to_vocab)

# convert integers back to words
decoded = []
for i in integer_encoded:
    decoded.append(int_to_vocab[i])
print(decoded[:100])
# check if the original text and decoded text match
print(words[:100])
print(decoded[:100])
# check if the original text and decoded text match
print(words == decoded)
if words != decoded:
    # show the differences between the original text and decoded text
    for i in range(len(words)):
        if words[i] != decoded[i]:
            print('Index: %s Original: %s, Decoded: %s' % (i, words[i], decoded[i]))
            
# save the vocabulary to file
vocab_filename = 'vocab.txt'
file = open(vocab_filename, 'wt')
file.write('\n'.join(vocab))
file.close()

30545
{'kurtarabilmek', '', 'yakalamıştı', 'samimi', 'koşun', 'pişmiş', 'uğramıyordu', 'adammış', 'alışırım', 'kısmetmiş', 'gülmekten', 'samanpazarı’ndan', 'cambazlıklar', 'gücenmiş', 'bitmemiştir', 'karanlıklar', 'büyüleyici', 'gibilerden', 'ödemeden', 'arabacının', 'kopmaya', 'bilmiyormuş', 'borç', 'devrilene', 'tarihinden', 'kadınsı', 'kırımlar', 'edecekleri', 'beyefendinin', 'kaçmaya', 'mükemmel', 'duyduğuna', 'beter', 'anlattığınız', 'belirtilerden', 'konmamıştı', 'örtüsüz', 'dayımın', 'modası', 'yolla', 'ciddiyet', 'tehlikenin', 'karşılaşmıştı', 'yetimlerini', 'vermekle', 'bağırılır', 'ısırmıştı', 'tefecilik', 'kurmaya', 'uyarıyorum', 'yayınlar', 'gibiler', 'çağına', 'kızcağızın', 'veriyorlardı', 'doğduğunu', 'ihtiyarın', 'gömleğin', 'evlerden', 'yer', 'gideceğim', 'gözlerinizle', 'yaptığı', 'gülümseyişle,', 'geçene', 'olacağımı', 'durduğum', 'misiniz?.', 'sırasında', 'hallerde', 'karşılaşıyoruz', 'kamburumsu', 'ürperir', 'seslendiğini', 'kolayını', 'şükrederim', 'nerelerdeydi', 

In [45]:
vocab[2]

'"'